# 02 – Prepare: Clean and Structure Fitbit Dataset

In [2]:
import pandas as pd
import glob
import os

csv_files = glob.glob("data_raw/extracted/**/*.csv", recursive=True)

raw_tables = {os.path.basename(f): pd.read_csv(f) for f in csv_files}


In [3]:
# Inspect each raw table
for name, df in raw_tables.items():
    print(f"--- {name} ---")
    print(df.shape)
    print(df.columns)
    print(df.isna().sum())
    print("\n")

--- dailyActivity_merged.csv ---
(940, 15)
Index(['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance',
       'LoggedActivitiesDistance', 'VeryActiveDistance',
       'ModeratelyActiveDistance', 'LightActiveDistance',
       'SedentaryActiveDistance', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories'],
      dtype='object')
Id                          0
ActivityDate                0
TotalSteps                  0
TotalDistance               0
TrackerDistance             0
LoggedActivitiesDistance    0
VeryActiveDistance          0
ModeratelyActiveDistance    0
LightActiveDistance         0
SedentaryActiveDistance     0
VeryActiveMinutes           0
FairlyActiveMinutes         0
LightlyActiveMinutes        0
SedentaryMinutes            0
Calories                    0
dtype: int64


--- heartrate_seconds_merged.csv ---
(2483658, 3)
Index(['Id', 'Time', 'Value'], dtype='object')
Id       0
Time     0
Value    0
dtyp

In [4]:
def clean_column_names(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

cleaned_tables = {name: clean_column_names(df) for name, df in raw_tables.items()}


In [11]:
import warnings

def convert_dates(df):
    df = df.copy()
    for col in df.columns:
        if any(keyword in col for keyword in ["date", "day", "time", "hour"]):
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                try:
                    df[col] = pd.to_datetime(df[col], errors="coerce")
                except:
                    pass
    return df

cleaned_tables = {name: convert_dates(df) for name, df in cleaned_tables.items()}


In [21]:
tables_to_keep = [
    "dailyactivity_merged.csv",
    "dailyintensities_merged.csv",
    "dailysteps_merged.csv",
    "dailycalories_merged.csv",
    "sleepday_merged.csv",
    "hourlysteps_merged.csv",
    "hourlycalories_merged.csv",
    "hourlyintensities_merged.csv"
]


In [23]:
tables_to_drop = [
    "weightloginfo_merged.csv",
    "heartrate_seconds_merged.csv",
    "minutecaloriesnarrow_merged.csv",
    "minuteintensitiesnarrow_merged.csv",
    "minutemetsnarrow_merged.csv",
    "minutesleep_merged.csv",
    "minutestepsnarrow_merged.csv",
    "minutecalorieswide_merged.csv",
    "minuteintensitieswide_merged.csv",
    "minutestepswide_merged.csv"
]


In [25]:
cleaned_tables = {
    os.path.basename(name).lower(): df
    for name, df in cleaned_tables.items()
}


In [27]:
analysis_tables = {name: cleaned_tables[name] for name in tables_to_keep}


In [29]:
for name, df in analysis_tables.items():
    df = df.copy()

    # Rename any date-like column to "date"
    if "activitydate" in df.columns:
        df.rename(columns={"activitydate": "date"}, inplace=True)
    if "activityday" in df.columns:
        df.rename(columns={"activityday": "date"}, inplace=True)
    if "sleepday" in df.columns:
        df.rename(columns={"sleepday": "date"}, inplace=True)

    analysis_tables[name] = df


In [31]:
for name, df in analysis_tables.items():
    before = df.shape[0]
    df = df.drop_duplicates()
    after = df.shape[0]
    print(f"{name}: removed {before - after} duplicates")
    analysis_tables[name] = df


dailyactivity_merged.csv: removed 0 duplicates
dailyintensities_merged.csv: removed 0 duplicates
dailysteps_merged.csv: removed 0 duplicates
dailycalories_merged.csv: removed 0 duplicates
sleepday_merged.csv: removed 3 duplicates
hourlysteps_merged.csv: removed 0 duplicates
hourlycalories_merged.csv: removed 0 duplicates
hourlyintensities_merged.csv: removed 0 duplicates


In [33]:
for name, df in analysis_tables.items():
    print(f"{name}: {df.shape}")
    print(df.head(), "\n")


dailyactivity_merged.csv: (940, 15)
           id       date  totalsteps  totaldistance  trackerdistance  \
0  1503960366 2016-04-12       13162           8.50             8.50   
1  1503960366 2016-04-13       10735           6.97             6.97   
2  1503960366 2016-04-14       10460           6.74             6.74   
3  1503960366 2016-04-15        9762           6.28             6.28   
4  1503960366 2016-04-16       12669           8.16             8.16   

   loggedactivitiesdistance  veryactivedistance  moderatelyactivedistance  \
0                       0.0                1.88                      0.55   
1                       0.0                1.57                      0.69   
2                       0.0                2.44                      0.40   
3                       0.0                2.14                      1.26   
4                       0.0                2.71                      0.41   

   lightactivedistance  sedentaryactivedistance  veryactiveminutes  

In [35]:
daily_activity = analysis_tables["dailyactivity_merged.csv"]
daily_intensities = analysis_tables["dailyintensities_merged.csv"]
daily_steps = analysis_tables["dailysteps_merged.csv"]
daily_calories = analysis_tables["dailycalories_merged.csv"]
daily_sleep = analysis_tables["sleepday_merged.csv"]

In [37]:
daily_merged = daily_activity.merge(
    daily_steps[["id", "date", "steptotal"]],
    on=["id", "date"],
    how="left"
)

In [39]:
daily_merged = daily_merged.merge(
    daily_calories[["id", "date", "calories"]],
    on=["id", "date"],
    how="left",
    suffixes=("", "_dailycal")
)


In [41]:
daily_merged = daily_merged.merge(
    daily_intensities,
    on=["id", "date"],
    how="left",
    suffixes=("", "_dailyint")
)


In [56]:
daily_merged.to_csv("data_clean/daily_merged.csv", index=False)


In [43]:
hourly_steps = analysis_tables["hourlysteps_merged.csv"]
hourly_calories = analysis_tables["hourlycalories_merged.csv"]
hourly_intensities = analysis_tables["hourlyintensities_merged.csv"]


In [45]:
for df in [hourly_steps, hourly_calories, hourly_intensities]:
    df.rename(columns={"activityhour": "datetime"}, inplace=True)


In [49]:
hourly_merged = hourly_steps.merge(
    hourly_calories, on=["id", "datetime"], how="left"
).merge(
    hourly_intensities, on=["id", "datetime"], how="left"
)


In [51]:
hourly_merged.shape
hourly_merged.head()
hourly_merged.info()
hourly_merged.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22099 entries, 0 to 22098
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                22099 non-null  int64         
 1   datetime          22099 non-null  datetime64[ns]
 2   steptotal         22099 non-null  int64         
 3   calories          22099 non-null  int64         
 4   totalintensity    22099 non-null  int64         
 5   averageintensity  22099 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 1.0 MB


id                  0
datetime            0
steptotal           0
calories            0
totalintensity      0
averageintensity    0
dtype: int64

In [53]:
import os
os.makedirs("data_clean", exist_ok=True)

hourly_merged.to_csv("data_clean/hourly_merged.csv", index=False)
